# Trying out the [goenrich](https://github.com/jdrudolph/goenrich) package for GO enrichment

In [1]:
%pylab inline
import numpy as np
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## II. Follow the [installation instructions](https://github.com/jdrudolph/goenrich#installation) for the goenrich package

```
pip install goenrich
```
```
mkdir db
```
#### Ontology
```
wget http://purl.obolibrary.org/obo/go/go-basic.obo -O db/go-basic.obo
```
#### UniprotACC
```
wget http://geneontology.org/gene-associations/goa_human.gaf.gz -O db/gene_association.goa_human.gaf.gz
```
#### Yeast SGD
```
wget http://downloads.yeastgenome.org/curation/literature/gene_association.sgd.gz -O db/gene_association.sgd.gz
```
#### Entrez GeneID
```
wget ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz -O db/gene2go.gz
```

## III. Try enrichment

In [4]:
import goenrich

# build the ontology
ontology_graph = goenrich.obo.ontology('db/go-basic.obo')
ontology_graph

In [5]:
# use all entrez geneid associations form gene2go as background
# use annot = goenrich.read.goa('db/gene_association.goa_human.gaf.gz') for uniprot
# use annot = goenrich.read.sgd('db/gene_association.sgd.gz') for yeast
gene2go = goenrich.read.gene2go('db/gene2go.gz')
# use values = {k: set(v) for k,v in annot.groupby('go_id')['db_object_symbol']} for uniprot/yeast
GO_terms_and_associated_genes = {k: set(v) for k,v in gene2go.groupby('GO_ID')['GeneID']}

In [6]:
# propagate the background through the ontology
background_set_attribute_name = 'genes'
goenrich.enrich.propagate(ontology_graph, GO_terms_and_associated_genes, background_set_attribute_name)

In [7]:
# extract some list of entries as example query
# use query = annot['db_object_symbol'].unique()[:20]
query = gene2go['GeneID'].unique()[:20]
query

array([ 1,  2,  9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28])

In [8]:
# for additional export to graphviz just specify the gvfile argument
# the show argument keeps the graph reasonably small
df = goenrich.enrich.analyze(ontology_graph, query, background_set_attribute_name)
df.head()

,M,N,n,name,namespace,p,q,rejected,term,x
0,19275,20,0,mitochondrion inheritance,biological_process,NaN,NaN,NaN,GO:0000001,0
1,19275,20,22,mitochondrial genome maintenance,biological_process,1.0,1.0,0.0,GO:0000002,0
2,19275,20,1348,reproduction,biological_process,NaN,NaN,NaN,GO:0000003,1
3,19275,20,0,high-affinity zinc uptake transmembrane transp...,molecular_function,NaN,NaN,NaN,GO:0000006,0
4,19275,20,0,low-affinity zinc ion transmembrane transporte...,molecular_function,NaN,NaN,NaN,GO:0000007,0


### It looks like goenrich wants entrezgene IDs -- could we fix that? 

## IV. Perform mapping so we can use gene symbols instead of entrezgene

In [9]:
import mygene
mg = mygene.MyGeneInfo()
df = mg.querymany(gene2go['GeneID'].unique().tolist(), scopes=['entrezgene'], fields=["HGNC", "symbol"], species="human", as_dataframe=True, returnall=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-19275...done.
Finished.


In [10]:
dup = df['dup']
missing = df['missing']
df = df['out']

In [11]:
dup, missing

([], [])

In [12]:
df.head()

,HGNC,_id,_score,symbol
query,,,,
1,5,1,23.310171,A1BG
2,7,2,23.312958,A2M
9,7645,9,23.949460,NAT1
10,7646,10,22.525192,NAT2
12,16,12,23.319983,SERPINA3


In [13]:
df = df[["symbol"]].reset_index()
df['query'] = pd.to_numeric(df['query'])
df.head()

,query,symbol
0,1,A1BG
1,2,A2M
2,9,NAT1
3,10,NAT2
4,12,SERPINA3


In [14]:
gene2go.head()

,tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category
534700,9606,1,GO:0002576,TAS,-,platelet degranulation,-,Process
534701,9606,1,GO:0003674,ND,-,molecular_function,-,Function
534702,9606,1,GO:0005576,IDA,-,extracellular region,3458201,Component
534703,9606,1,GO:0005576,TAS,-,extracellular region,-,Component
534704,9606,1,GO:0005615,IDA,-,extracellular space,16502470,Component


In [18]:
gene2go = gene2go.merge(df, how='left', left_on='GeneID', right_on='query')
del gene2go['query']
gene2go = gene2go.rename(columns={'symbol':'GeneSymbol'})
gene2go.head()

,tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category,GeneSymbol
0,9606,1,GO:0002576,TAS,-,platelet degranulation,-,Process,A1BG
1,9606,1,GO:0003674,ND,-,molecular_function,-,Function,A1BG
2,9606,1,GO:0005576,IDA,-,extracellular region,3458201,Component,A1BG
3,9606,1,GO:0005576,TAS,-,extracellular region,-,Component,A1BG
4,9606,1,GO:0005615,IDA,-,extracellular space,16502470,Component,A1BG


In [19]:
gene2go.to_csv('gene2go.csv', header=True, index=False)

## V. Try goenrich with updated GO database

In [1]:
import goenrich

# build the ontology
ontology_graph = goenrich.obo.ontology('db/go-basic.obo')
ontology_graph

In [4]:
gene2go = pd.read_csv('gene2go.csv')
gene2go.head()

,tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category,GeneSymbol
0,9606,1,GO:0002576,TAS,-,platelet degranulation,-,Process,A1BG
1,9606,1,GO:0003674,ND,-,molecular_function,-,Function,A1BG
2,9606,1,GO:0005576,IDA,-,extracellular region,3458201,Component,A1BG
3,9606,1,GO:0005576,TAS,-,extracellular region,-,Component,A1BG
4,9606,1,GO:0005615,IDA,-,extracellular space,16502470,Component,A1BG


In [5]:
GO_terms_and_associated_genes = {k: set(v) for k,v in gene2go.groupby('GO_ID')['GeneSymbol']}

In [6]:
# propagate the background through the ontology
background_set_attribute_name = 'genes'
goenrich.enrich.propagate(ontology_graph, GO_terms_and_associated_genes, background_set_attribute_name)

In [9]:
query = clustering[clustering.level1 == 3]['name'].tolist()
query[:5]

['RNF11', 'TRIM46', 'TIMM8A', 'GLB1', 'ESPL1']

In [19]:
df = goenrich.enrich.analyze(ontology_graph, query, background_set_attribute_name).dropna().sort_values('p')
df.head()

,M,N,n,name,namespace,p,q,rejected,term,x
3486,19275,41,79,thiol-dependent ubiquitin-specific protease ac...,molecular_function,2.201579e-26,1.636653e-22,1.0,GO:0004843,14
18548,19275,41,121,thiol-dependent ubiquitinyl hydrolase activity,molecular_function,2.090338e-23,5.898496e-20,1.0,GO:0036459,14
35783,19275,41,122,ubiquitinyl hydrolase activity,molecular_function,2.380345e-23,5.898496e-20,1.0,GO:0101005,14
20505,19275,41,495,modification-dependent macromolecule catabolic...,biological_process,4.396737e-23,8.171335e-20,1.0,GO:0043632,20
5740,19275,41,197,cysteine-type peptidase activity,molecular_function,6.297426e-22,9.363012e-19,1.0,GO:0008234,15
